In [1]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools
import numpy as np

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Process mutation files

In [2]:
mutfile = pd.read_csv("patient01_genotype_sbs.txt", sep="\t")
patient = "patient01"
l = []
for _,r in mutfile.iterrows():
    micros = ",".join(r[r>0].index)
    mutid = r.name
    chrom = mutid.split(":")[0]
    pos = int(mutid.split(":")[1])
    ref,alt = mutid.split(":")[2:]
    mtype = "SBS" if len(ref)==1 else "Other"
    l.append([patient, micros, np.nan, mutid, chrom, pos, ref, alt, mtype])

df = pd.DataFrame(l, columns=['PatientID', 'MicrobiopsyID', 'ClusterID', 'MutationID', 'Chr',
       'Pos_hg38', 'Ref', 'Alt', 'type'])

dat = pd.read_excel("41588_2023_1545_MOESM4_ESM.xlsx", sheet_name="TableS4_all_mutations")
dat = dat[(dat.type=="SBS") & (dat.PatientID=="patient01")].reset_index(drop=True)

cols = ["PatientID", "MicrobiopsyID", "Chr", "Pos_hg38", "Ref", "Alt"]
dat = dat[cols].sort_values(by=["Chr", "Pos_hg38"]).reset_index(drop=True)
df  = df[cols].sort_values(by=["Chr", "Pos_hg38"]).reset_index(drop=True)
### This process generates a dataframe that is exactly the same as TableS4
df.equals(dat)

True

In [3]:
path = "binomial_filtering/binomial_filters"
l = []

for patient in os.listdir(path):
    if "patient" not in patient: continue
    mutdata = pd.read_table(f"{path}/{patient}/{patient}_genotype_sbs.txt")
    for _,r in mutdata.iterrows():
        micros = r[r>0].index
        mutid = r.name
        chrom = mutid.split(":")[0]
        pos = int(mutid.split(":")[1])
        ref,alt = mutid.split(":")[2:]
        mtype = "SBS" if len(ref)==1 else "Other"
        for micro in micros:
            l.append([patient, micro, mutid, chrom, pos, ref, alt, mtype])
dat = pd.DataFrame(l, columns=['Individual', 'Sample', 'mutationid', 
                               'Chr', 'Pos', 'Ref', 'Alt', 'type'])

In [4]:
len(dat.Individual.unique())

115

In [5]:
dat["Compartment"] = "WES+Bulk"
dat["Tissue_CellType"] = "Skin"
dat["Study"] = "Olafsson2023"
dat = dat[dat.type=="SBS"]

In [6]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg38.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [10]:
dat["Compartment"] = dat.apply(lambda row: row.Compartment if "WGS" not in row.Individual else "WGS", axis=1)
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]
dat[col_order].to_csv("Olafsson2023.csv.gz", compression="gzip", index=False)

In [130]:
dat[~dat.Individual.str.contains("WG")].Compartment.unique()

array(['WES+Bulk'], dtype=object)

In [111]:
sdat = pd.read_csv("Olafsson2023.csv.gz")

In [123]:
sdat[sdat.Sample.str.contains("P34H_LL_3")]

,Chr,Pos,Ref,Alt,Sample,Tissue_CellType,Compartment,Individual,Study,Context,GeneralType,Type
271570,chr10,100012694,A,G,P34H_LL_3,Skin,WES+Bulk,patient34_WGS,Olafsson2023,AAC,A[A>G]C,G[T>C]T
271573,chr10,100040322,A,C,P34H_LL_3,Skin,WES+Bulk,patient34_WGS,Olafsson2023,TAC,T[A>C]C,G[T>G]A
271574,chr10,100062252,T,G,P34H_LL_3,Skin,WES+Bulk,patient34_WGS,Olafsson2023,ATA,A[T>G]A,A[T>G]A
271577,chr10,100181105,A,T,P34H_LL_3,Skin,WES+Bulk,patient34_WGS,Olafsson2023,TAT,T[A>T]T,A[T>A]A
271581,chr10,100204983,A,T,P34H_LL_3,Skin,WES+Bulk,patient34_WGS,Olafsson2023,TAC,T[A>T]C,G[T>A]A
...,...,...,...,...,...,...,...,...,...,...,...,...
837723,chr9,99074014,A,G,P34H_LL_3,Skin,WES+Bulk,patient34,Olafsson2023,TAA,T[A>G]A,T[T>C]A
837724,chr9,99137754,T,A,P34H_LL_3,Skin,WES+Bulk,patient34,Olafsson2023,ATA,A[T>A]A,A[T>A]A
837726,chr9,99227841,T,A,P34H_LL_3,Skin,WES+Bulk,patient34,Olafsson2023,ATA,A[T>A]A,A[T>A]A
837739,chr9,99909774,T,A,P34H_LL_3,Skin,WES+Bulk,patient34,Olafsson2023,ATA,A[T>A]A,A[T>A]A


Read data

In [1]:
dat = pd.read_excel("41588_2023_1545_MOESM4_ESM.xlsx", sheet_name="TableS4_all_mutations")

In [33]:
dat[(dat.PatientID=="patient01") & (dat.type=="SBS")]

,PatientID,MicrobiopsyID,ClusterID,MutationID,Chr,Pos_hg38,Ref,Alt,type
0,patient01,"P01L_LL_10,P01L_17",patient01_36,P01:chr1:10018498:A:T,chr1,10018498,A,T,SBS
1,patient01,P01L_16,patient01_4,P01:chr1:100268476:A:C,chr1,100268476,A,C,SBS
2,patient01,P01L_LL_13,patient01_28,P01:chr1:100348186:G:A,chr1,100348186,G,A,SBS
3,patient01,P01L_20,patient01_38,P01:chr1:100743353:G:A,chr1,100743353,G,A,SBS
4,patient01,P01H_LL_8,patient01_20,P01:chr1:1007997:C:T,chr1,1007997,C,T,SBS
...,...,...,...,...,...,...,...,...,...
15231,patient01,P01L_LL_12,patient01_30,P01:chrX:9809778:C:T,chrX,9809778,C,T,SBS
15232,patient01,P01H_LL_6,patient01_24,P01:chrX:9836549:C:T,chrX,9836549,C,T,SBS
15233,patient01,"P01L_LL_10,P01L_LL_12",NaN,P01:chrY:19716089:T:C,chrY,19716089,T,C,SBS
15234,patient01,P01L_18,NaN,P01:chrY:20781363:A:T,chrY,20781363,A,T,SBS


In [3]:
dat["Individual"] = dat.PatientID
dat["Pos"] = dat.Pos_hg38
dat["Sample"] = dat["ClusterID"]
dat["Compartment"] = "WES+Bulk"
dat["Tissue_CellType"] = "Skin"
dat["Study"] = "Olafsson2023"
dat = dat[dat.type=="SBS"]

In [4]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg38.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [22]:
ndat_l = []
for _,r in dat.iterrows():
    for microb in r.MicrobiopsyID.split(","):
        r.Sample = microb
        ndat_l.append(r)
ndat = pd.DataFrame(ndat_l)

In [25]:
ndat[col_order].to_csv("Olafsson2023.csv.gz", compression="gzip", index=False)

In [1]:
pmeta1 = pd.read_excel("41588_2023_1545_MOESM4_ESM.xlsx", sheet_name="TableS1_patient_metadata")
pmeta2 = pd.read_excel("41588_2023_1545_MOESM4_ESM.xlsx", sheet_name="TableS2_Microdissection_metadat")

In [4]:
pmeta2.columns

Index(['SampleID', 'BiopsyID', 'MetaLocation', 'DetailedLocation',
       'DiseaseStatus', 'PatientID', 'SangerSampleID', 'SangerPlateID',
       'CutVolume', 'CutArea', 'NrZstacks', 'LibraryConcentration',
       'ASCAT_normal', 'MedianCoverage', 'MedianVAF', 'PCT_PF_UQ_READS',
       'PCT_OFF_BAIT', 'FreeMix', 'ExclusionCriteria',
       'TotalNumberSubstitutions', 'TotalNumberSingleBase',
       'TotalNumberDoubleBase', 'TotalNumberIndels', 'SubsGtVAF10',
       'SbsGtVAF10', 'DbsGtVAF10', 'IndelGtVAF10'],
      dtype='object')

In [21]:
cell2indv = dat.set_index("Sample")["PatientID"].to_dict()
ages = pmeta1.set_index("Patient.ID")["Age_at_sampling"].to_dict()
stat = pmeta2.set_index("PatientID")["DiseaseStatus"].to_dict()
c = {"Non-lesional":"Normal", "Lesional":"Psoriasis"}
stat = {k:c[v] for k,v in stat.items()}

In [26]:
pmeta2.columns

Index(['SampleID', 'BiopsyID', 'MetaLocation', 'DetailedLocation',
       'DiseaseStatus', 'PatientID', 'SangerSampleID', 'SangerPlateID',
       'CutVolume', 'CutArea', 'NrZstacks', 'LibraryConcentration',
       'ASCAT_normal', 'MedianCoverage', 'MedianVAF', 'PCT_PF_UQ_READS',
       'PCT_OFF_BAIT', 'FreeMix', 'ExclusionCriteria',
       'TotalNumberSubstitutions', 'TotalNumberSingleBase',
       'TotalNumberDoubleBase', 'TotalNumberIndels', 'SubsGtVAF10',
       'SbsGtVAF10', 'DbsGtVAF10', 'IndelGtVAF10'],
      dtype='object')

In [23]:
l = list(dat.Sample.unique())
meta = pd.DataFrame(l)
meta.columns = ["Sample"]
meta["Study"] = "Olafsson2023"
meta["Individual"] = meta.Sample.map(cell2indv)
meta["Tissue_CellType"] = "Skin"
meta["Age"] = meta["Individual"].map(ages)
meta["Status"] = meta.Sample.str.split(",").str[0].map(stat)
meta["Compartment"] = "WES+Bulk"

In [27]:
meta[col_meta].to_csv("Olafsson2023.tab.gz",sep="\t", index=False)